*In this notebook:*<br>
CRISP-DM: Step 3<br>
**Preparing the Data** for the next step
___

# The Host Perspective

In notebook 02 we had a look at the first two questions:

1. Which parameters influence a listings price?
1. What parameter can a host use to improve price and value?

To answer the third question:<br>

**Can we make a good price estimation for a new offer to assist a (new) host?**<br>

We will try to make predictions using a machine learning algorithm. To do that, we need to finish what we started in notebook 01 and clean the data.

In [2]:
import numpy as np
import pandas as pd

from ExploreData import sort_mean, value_counter, index_by_key
#from TransformData import \
#    price_transform, rate_transform, split_column_values, date_transform

import CategoricalPrep as cp
import FeatureEngine as fe
#import PrepAndModel as pam

# Import the data
df_listings = pd.read_csv('./data/listings.csv')


## Cleaning the Data Set
After analyzing the data in detail, additional steps are necessary to prepare the data for machine learning. Some cleaning has already been done before hand, to better analyze the data. The rest comes now:

* There are still categorical columns which are a problem for ml algorithms
* Up to now missing values have not been dealt with - this is also a Problem for ml algorithms

But first come the preparations that are already implemented:

In [3]:
# Drop Columns without information
listings_drop_col = cp.drop_columns_analyze(df_listings)
# Transform categorical columns and make information accessable
listings_drop_trans = cp.transform_columns(listings_drop_col)
# Create new price features
listings_features = fe.new_features(listings_drop_trans)

In [4]:
df = listings_features
df.shape

(3818, 126)

In [5]:
df_values = value_counter(df)
df_values.head()

,val_count,nan_count,val_pcnt,nan_pcnt,col_dtype
id,3818,0,100.000000,0.000000,int64
name,3792,0,99.319015,0.000000,object
summary,3478,177,91.094814,4.635935,object
space,3119,569,81.691985,14.903091,object
description,3742,0,98.009429,0.000000,object


### 1. What to do with the remaining categorical columns?
As the cell above shows, there are still object columns remaining. To deal with them a closer look is required, since many columns just contain descriptive text and can not be used to train a model without additional steps.
* There are columns that will be droped
* And there columns which need proper encoding

The cell below shows that text columns can be identified by the many different values they hold

In [6]:
# Selcet only object columns
df_categorical = df.select_dtypes(include=['object'])
# Update df_values
df_values = value_counter(df_categorical)
df_values

,val_count,nan_count,val_pcnt,nan_pcnt,col_dtype
name,3792,0,99.319015,0.000000,object
summary,3478,177,91.094814,4.635935,object
space,3119,569,81.691985,14.903091,object
description,3742,0,98.009429,0.000000,object
neighborhood_overview,2506,1032,65.636459,27.029859,object
notes,1999,1606,52.357255,42.063908,object
transit,2574,934,67.417496,24.463070,object
host_about,2011,859,52.671556,22.498690,object
host_neighbourhood,102,300,2.671556,7.857517,object
street,1442,0,37.768465,0.000000,object


In [6]:
# Select attributes with 100 or less uniqu values
df_few_values = df_values[df_values['val_count']<=100]
df_few_values

,val_count,nan_count,val_pcnt,nan_pcnt,col_dtype
host_response_time,4,523,0.104767,13.698271,object
neighbourhood_group_cleansed,17,0,0.445259,0.000000,object
zipcode,28,7,0.733368,0.183342,object
property_type,16,1,0.419068,0.026192,object
room_type,3,0,0.078575,0.000000,object
bed_type,5,0,0.130959,0.000000,object


These columns can all be encoded with 01-encoding.
* The values are not ordinal
* There are not too many different values per column

It seems columns with ordinal information have all already been dealt with by the `transform_columns()` function.<br>
By selecting columns with  equal to or less than 100 values some columns that do not contain text are left out. But by using 01-encoding the number of columns is increased by the number of values. <br>
To keep the number of columns from exploding this is accepted for the moment. We can always come back, if necessary, and change it.

In [7]:
# Before creating dummy columns look at the zipcode and fix it:
df['zipcode'].value_counts()

98122        420
98103        394
98102        339
98105        225
98109        202
98101        201
98144        200
98121        196
98107        180
98115        178
98112        167
98117        164
98118        154
98119        143
98116        112
98104         95
98125         71
98199         66
98126         63
98106         58
98108         56
98133         46
98136         44
98177         19
98178          7
98134          5
98146          5
99\n98122      1
Name: zipcode, dtype: int64

In [8]:
# Fix it:
df.loc[df['zipcode']=='99\n98122', 'zipcode'] = '98122'

In [9]:
# Create dummy columns
few_values = df_few_values.index
df_dummies = pd.get_dummies(
    df[few_values],
    prefix_sep='_',
    drop_first=True,
    dummy_na=True
    )

df_dummies.head()

,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,neighbourhood_group_cleansed_Beacon Hill,neighbourhood_group_cleansed_Capitol Hill,neighbourhood_group_cleansed_Cascade,neighbourhood_group_cleansed_Central Area,neighbourhood_group_cleansed_Delridge,neighbourhood_group_cleansed_Downtown,...,property_type_Yurt,property_type_nan,room_type_Private room,room_type_Shared room,room_type_nan,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,bed_type_nan
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Add dummy columns to the data set and drop the original categorical columns
df = df.join(df_dummies)
df = df.drop(few_values, axis = 1)

This leaves the rest of the categorical columns which have already been declared to have too many values. Thus they are droped from the data set completely.

In [11]:
# Select attributes with more than 100 values and drop them
to_many_values = df_values[df_values['val_count']>100].index
df = df.drop(to_many_values, axis=1)

In [12]:
# Check the data set
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 176 columns):
 #    Column                                            Dtype  
---   ------                                            -----  
 0    id                                                int64  
 1    host_id                                           int64  
 2    host_response_rate                                float64
 3    host_acceptance_rate                              float64
 4    host_is_superhost                                 float64
 5    host_listings_count                               float64
 6    host_total_listings_count                         float64
 7    host_has_profile_pic                              float64
 8    host_identity_verified                            float64
 9    latitude                                          float64
 10   longitude                                         float64
 11   is_location_exact                                 int6

### 2. What to do about the missing values?
Again, a closer look to assess the data is necessary to pick a strategy. <br>
There are not too many missing values in the remaining columns. The cell below shows the percentage per column. Only `security_deposit` has more than 50% missing values. Another important fact is, many of the columns below have very few discrete values. Replacing missing values with a mean might be the wrong approach here.

* Identify features where missing values can have a meaning ( no review_score or no security_deposit)<br>
    store the information in an extra column
* Columns with few values and few missing values are filled up with the mode
* The remaining columns are treated with a mean value

In [7]:
# Update df_values
df_values = value_counter(df)
# Identify columns with missing values
df_nans = df_values[df_values['nan_pcnt'] > 0]
nans_attributes = df_nans.index

In [9]:
# Create extra columns for nan values and inpute 0 in the original column
col_names = index_by_key(df, ['review_scores', 'security'])

if 'has_review_scores_rating' not in col_names:
    for col in col_names:
        new_name = 'has_' + col
        df[new_name] = df[col].notnull().astype('int')
        df[col] = df[col].fillna(0)

# Remove the names from the list
nans_attributes = nans_attributes.difference(col_names)

df[index_by_key(df, ['review_scores', 'security'])].head()

,security_deposit,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,has_security_deposit,has_review_scores_rating,has_review_scores_accuracy,has_review_scores_cleanliness,has_review_scores_checkin,has_review_scores_communication,has_review_scores_location,has_review_scores_value
0,0.0,95.0,10.0,10.0,10.0,10.0,9.0,10.0,0,1,1,1,1,1,1,1
1,100.0,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1,1,1,1,1,1,1,1
2,1000.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1,1,1,1,1,1,1,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0
4,700.0,92.0,9.0,9.0,10.0,10.0,9.0,9.0,1,1,1,1,1,1,1,1


In [15]:
# Host_response_rate: Fill nan values with the mean
df['host_response_rate'] = df['host_response_rate'].fillna(df['host_response_rate'].mean())

# Remove the name from the list
nans_attributes = nans_attributes.difference(['host_response_rate'])


In [16]:
# The remaining columns can be filled with the mode
df_mode = df[nans_attributes]
df_mode = df_mode.fillna(df_mode.mode().squeeze(), axis=0)
df[nans_attributes] = df_mode

df[nans_attributes].head()

,bathrooms,bedroom_price,bedrooms,beds,cleaning_fee,host_acceptance_rate,host_has_profile_pic,host_identity_verified,host_is_superhost,host_listings_count,host_since_day,host_since_month,host_since_year,host_total_listings_count,reviews_per_month
0,1.0,85.0,1.0,1.0,50.0,100.0,1.0,1.0,0.0,3.0,11.0,8.0,2011.0,3.0,4.07
1,1.0,150.0,1.0,1.0,40.0,100.0,1.0,1.0,1.0,6.0,21.0,2.0,2013.0,6.0,1.48
2,4.5,195.0,5.0,7.0,300.0,100.0,1.0,1.0,0.0,2.0,12.0,6.0,2014.0,2.0,1.15
3,1.0,100.0,0.0,2.0,50.0,100.0,1.0,1.0,0.0,1.0,6.0,11.0,2013.0,1.0,1.00
4,2.0,150.0,3.0,3.0,125.0,100.0,1.0,1.0,0.0,2.0,29.0,11.0,2011.0,2.0,0.89
